# Read a STAC Catalog Using PySTAC

In this tutorial, you will gain an understanding of how to explore a STAC Catalog using [PySTAC](https://pystac.readthedocs.io/en/stable/index.html). You will be able to use these skills to explore any of the [many existing STAC Catalogs](https://stacindex.org/catalogs).

This tutorial as well as the following 3 tutorials were adapted from the [PySTAC site tutorials](https://pystac.readthedocs.io/en/stable/index.html).

Throughout this tutorial, a variety of PySTAC classes, methods, and instances are used to read the STAC Catalog. We encourage you to look at the [PySTAC API Reference](https://pystac.readthedocs.io/en/latest/api.html) while going through this tutorial.

## Dependencies
If you need to install pystac, uncomment the line below and run this cell.

In [ ]:
# ! pip install pystac

## STAC Catalogs
[A STAC Catalog](https://github.com/radiantearth/stac-spec/tree/master/catalog-spec) is used to group other STAC objects like items, collections, and/or even other catalogs.

We will be using a small example catalog adapted from the [example Landsat Collection](https://github.com/geotrellis/geotrellis-server/tree/977bad7a64c409341479c281c8c72222008861fd/stac-example/catalog/landsat-stac-collection) in the [GeoTrellis](https://geotrellis.io) repository. All STAC Items and Collections can be found in the [docs/example-catalog](https://github.com/stac-utils/pystac/tree/main/docs/example-catalog) directory of the PySTAC documentation repository; all assets are hosted in the Landsat S3 bucket.

#### Import Packages and Store Data
To begin, import the packages and PySTAC classes that you need to access data and work with STAC Catalogs in Python.

In [ ]:
import json

from pystac import Catalog, get_stac_version
from pystac.extensions.eo import EOExtension
from pystac.extensions.label import LabelExtension

In [ ]:
# Read the example catalog
root_catalog = Catalog.from_file('https://raw.githubusercontent.com/stac-utils/pystac/main/docs/example-catalog/catalog.json')

## Explore the High-Level Catalog Information

To give us an idea of how the catalog we are working with is organized, let's take a look at all elements of the STAC using the `describe` method.

<div style="background-color:rgba(181, 216, 229, 0.8); text-align:center; vertical-align: top; padding:10px; ml:5px; 0;">
NOTE: Be careful using the `describe` method on large catalogs, as it will walk and print the entire tree of the STAC.
</div>

In [ ]:
root_catalog.describe()

* <Catalog id=landsat-stac-collection-catalog>
    * <Collection id=landsat-8-l1>
      * <Item id=LC80140332018166LGN00>
      * <Item id=LC80150322018141LGN00>
      * <Item id=LC80150332018189LGN00>
      * <Item id=LC80300332018166LGN00>


From this output, we can see that the catalog has 1 collection and that this collection has 4 items.

Now, let's look at the root catalog more in depth.

In [ ]:
# Print some basic metadata from the Catalog
print(f"ID: {root_catalog.id}")
print(f"Title: {root_catalog.title or 'N/A'}")
print(f"Description: {root_catalog.description or 'N/A'}")

ID: landsat-stac-collection-catalog
Title: STAC for Landsat data
Description: STAC for Landsat data


<div style="background-color:rgba(181, 216, 229, 0.8); text-align:center; vertical-align: top; padding:10px; ml:5px; 0;">
Note: we do not print the "stac_version" here. PySTAC automatically updates any catalogs to the most recent supported STAC version and will automatically write this to the JSON object during serialization.
</div>  


Let's confirm the latest STAC Specification version supported by PySTAC.

In [ ]:
print(get_stac_version())

1.0.0


With this information, we have an understanding of the layout and general information about the STAC Catalog at hand. Now, let's dive deeper into this catalog.

## Crawl STAC Child Catalogs and/or Collections

[STAC Collections](https://github.com/radiantearth/stac-spec/tree/master/collection-spec) are used to group related items and provide aggregate or summary metadata for those items.

STAC Catalogs may have many nested layers of catalogs or collections within the top-level collection. Our example catalog only has one collection within the main catalog at [landsat-8-l1/collection.json](/public/example-catalog/landsat-8-l1/collection.json). We can list the collections in a given catalog using the `Catalog.get_collections` method. This method returns an iterable of PySTAC [Collection](https://pystac.readthedocs.io/en/latest/api.html#collection) instances, which we will turn into a `list`.

In [ ]:
collections = list(root_catalog.get_collections())

print(f"Number of collections: {len(collections)}")
print("Collections IDs:")
for collection in collections:
    print(f"- {collection.id}")

Number of collections: 1
Collections IDs:
- landsat-8-l1


Let's grab that collection as a PySTAC [Collection](https://pystac.readthedocs.io/en/latest/api/collection.html#pystac-collection) instance using the `Catalog.get_child` method so we can look at it in more detail. This method gets a child catalog or collection by ID, so we'll use the collection ID that we printed above. Since this method returns `None` if no child exists with the given ID, let's check to make sure we actually got the `Collection`.

In [ ]:
collection = root_catalog.get_child("landsat-8-l1")
if collection is None:
    print("Collection is Empty. Check your downloads and try again.")
else:
    print("Collection has a root child. You may proceed to the following steps.")

Collection has a root child. You may proceed to the following steps.


## Crawl the STAC Items

[STAC Items](https://github.com/radiantearth/stac-spec/tree/master/item-spec) are the fundamental building blocks of a STAC Catalog. Each Item represents a single spatiotemporal resource (e.g. a satellite scene).

Both catalogs and collections may have items associated with them. Let's crawl our catalog, starting at the root, to see what Items we have. The `Catalog.get_all_items` method provides a convenient way of recursively listing all Items associated with a catalog and all of its sub-catalogs.

In [ ]:
items = list(root_catalog.get_all_items())

print(f"Number of items: {len(items)}")
for item in items:
    print(f"- {item.id}")

Number of items: 4
- LC80140332018166LGN00
- LC80150322018141LGN00
- LC80150332018189LGN00
- LC80300332018166LGN00


These IDs are not very descriptive; in the next section, we will see how we can access the rich metadata associated with each item.

## Explore STAC Item Metadata

### Item Metadata

Items can have *a lot* of metadata. This can be a bit overwhelming at first, but let's break the metadata fields down into a few categories:

1. Core Item Metadata
2. Common Metadata
3. STAC Extensions

We will walk through each of these metadata categories in the following sections.

First, let's grab one of the Items using the `Catalog.get_item` method. We will use `recursive=True` to recursively crawl all child catalogs and/or collections to find the item.

In [ ]:
item = root_catalog.get_item("LC80140332018166LGN00", recursive=True)

#### 1. Core Item Metadata

The core item metadata fields include spatiotemporal information and the ID of the collection to which the item belongs. These fields are all at the top level of the item JSON and we can access them through attributes on the [PySTAC Item](https://pystac.readthedocs.io/en/latest/api/collection.html#pystac-collection) instance.

In [ ]:
item.geometry

{'type': 'Polygon',
 'coordinates': [[[-76.12180471942207, 39.95810181489563],
   [-73.94910518227414, 39.55117185146004],
   [-74.49564725552679, 37.826064511480496],
   [-76.66550404911956, 38.240699151776084],
   [-76.12180471942207, 39.95810181489563]]]}

In [ ]:
item.bbox

[-76.66703, 37.82561, -73.94861, 39.95958]

In [ ]:
item.datetime

datetime.datetime(2018, 6, 15, 15, 39, 9, tzinfo=tzutc())

In [ ]:
item.collection_id

'landsat-8-l1'

We can also access the same information as the cell above by running the `Item.get_collection` method.

In [ ]:
item.get_collection()

<Collection id=landsat-8-l1>

####  2. Common Metadata

Certain fields that are commonly used in Items, but may also be found in other objects (e.g. Assets) are defined in the [Common Metadata](https://github.com/radiantearth/stac-spec/blob/v1.0.0/item-spec/common-metadata.md) section of the spec. These include licensing and instrument information, descriptions of datetime ranges, and some other common fields. These properties can be found as attributes of the `Item.common_metadata` property, which is an instance of the `CommonMetadata` class.

In [ ]:
item.common_metadata.instruments

['OLI_TIRS']

In [ ]:
item.common_metadata.platform

'landsat-8'

In [ ]:
item.common_metadata.gsd

30

#### 3. STAC Extensions

[STAC Extensions](https://github.com/radiantearth/stac-spec/tree/master/extensions) are a mechanism for providing additional metadata not covered by the core STAC Spec. We can see which STAC Extensions are implemented by this particular Item by examining the list of extension URIs in the `stac_extensions` field.

In [ ]:
item.stac_extensions

['https://stac-extensions.github.io/eo/v1.1.0/schema.json',
 'https://stac-extensions.github.io/view/v1.0.0/schema.json',
 'https://stac-extensions.github.io/projection/v1.1.0/schema.json']

This Item implements the [Electro-Optical](https://github.com/stac-extensions/eo), [View Geometry](https://github.com/stac-extensions/view), and [Projection](https://github.com/stac-extensions/projection) Extensions.

We can also check if a specific extension is implemented using the `has_extension` method for that extension class.

In [ ]:
EOExtension.has_extension(item)

True

In [ ]:
LabelExtension.has_extension(item)

False

We can access fields associated with the extension as attributes on the extension instance. For instance, the ["eo:cloud_cover" field](https://github.com/stac-extensions/eo#item-properties-or-asset-fields) defined in the Electro-Optical Extension can be accessed using the [EOExtension.cloud_cover](https://pystac.readthedocs.io/en/latest/api.html#pystac.extensions.eo.EOExtension.cloud_cover) attribute.

In [ ]:
eo_item_ext = EOExtension.ext(item)
eo_item_ext.cloud_cover

22

We can also access the cloud cover field directly in the Item properties.

In [ ]:
item.properties['eo:cloud_cover']

22

## Access STAC Item's Assets

To access the item's assets, we can use the `assets` attribute, which is a dictionary:

In [ ]:
for asset_key in item.assets:
    asset = item.assets[asset_key]
    print('{}: {} ({})'.format(asset_key, asset.href, asset.media_type))

index: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/index.html (text/html)
thumbnail: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_thumb_large.jpg (image/jpeg)
B1: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_B1.TIF (image/tiff)
B2: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_B2.TIF (image/tiff)
B3: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_B3.TIF (image/tiff)
B4: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_B4.TIF (image/tiff)
B5: https://s3-us

We can use the `to_dict()` method to convert an asset, or any PySTAC object, into a dictionary:

In [ ]:
asset = item.assets['B3']
asset.to_dict()

{'href': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_B3.TIF',
 'type': 'image/tiff',
 'title': 'Band 3 (green)',
 'eo:bands': [{'name': 'B3',
   'full_width_half_max': 0.06,
   'center_wavelength': 0.56,
   'common_name': 'green'}],
 'roles': []}

Here, we use the `eo Extension` to get the band information for the asset:

In [ ]:
eo_asset_ext = EOExtension.ext(asset)
bands = eo_asset_ext.bands
bands

[<Band name=B3>]

In [ ]:
bands[0].to_dict()

{'name': 'B3',
 'full_width_half_max': 0.06,
 'center_wavelength': 0.56,
 'common_name': 'green'}

You have now successfully explored the common components of an existing STAC Catalog. To learn how to write your own STAC Catalog, see the [following tutorial](/en/tutorials/2-create-stac-catalog-python/index.html).

#### Join the conversation
If you have any questions, you’re welcome to ask our community on [CNG Discourse](https://cng.discourse.group/c/stac/15).

In [1]:
! pip install terracatalogueclient

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.4/92.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 7.0 MB/s eta 0:00:00


In [2]:
from terracatalogueclient import Catalogue
catalogue = Catalogue()

In [5]:
from terracatalogueclient import Catalogue
catalogue = Catalogue().authenticate_non_interactive("aquatics", "Welcome!")

In [6]:
collections = catalogue.get_collections()
for c in collections:
    print(f"{c.id} - {c.properties['title']}")

urn:eop:VITO:CGS_S1_GRD_L1 - Sentinel-1 Level-1 Ground Range Detected (GRD) products
urn:eop:VITO:CGS_S1_GRD_SIGMA0_L1 - Sentinel-1 Level-1 Ground Range Detected (GRD) SIGMA0 products
urn:eop:VITO:CGS_S1_SLC_L1 - Sentinel-1 Level-1 Single Look Complex (SLC) products.
urn:eop:VITO:CGS_S2_L1C - Sentinel-2 Level-1C MSI product contains Top-of-atmosphere reflectances in cartographic geometry
urn:eop:VITO:COP_DEM_EU_25M - Copernicus European Digital Elevation Model (EU-DEM), version 1.1 in 25m resolution
urn:eop:VITO:COP_DEM_GLO_30M_COG - Copernicus Global 30 meter Digital Elevation Model dataset in COG format  
urn:eop:VITO:COP_DEM_GLO_90M_COG - Copernicus Global 90 meter Digital Elevation Model dataset in COG format
urn:eop:VITO:ESA_WORLDCEREAL_ACTIVECROPLAND_V1 - ESA WorldCereal products for active cropland per agro-ecological zone - V1
urn:eop:VITO:ESA_WORLDCEREAL_IRRIGATION_V1 - ESA WorldCereal products for active irrigation per agro-ecological zone - V1
urn:eop:VITO:ESA_WORLDCEREAL_MA

Query products:

In [23]:
import datetime as dt
products = catalogue.get_products(
    "urn:eop:VITO:TERRASCOPE_S2_NDVI_V2",
    start=dt.date(2021, 5, 1),
    end=dt.date(2021, 6, 1),
    tileId="31UGS",
    accessedFrom="HTTP"
)
for product in products:
    print(product.title)
    for data in product.data:
      print(data.href)
#products_list = list(products)

S2A_20210502T104021_31UGS_NDVI_10M_V210
https://services.terrascope.be/download/Sentinel2/NDVI_V2/2021/05/02/S2A_20210502T104021_31UGS_NDVI_V210/S2A_20210502T104021_31UGS_NDVI_10M_V210.tif
S2A_20210505T105031_31UGS_NDVI_10M_V200
https://services.terrascope.be/download/Sentinel2/NDVI_V2/2021/05/05/S2A_20210505T105031_31UGS_NDVI_V200/S2A_20210505T105031_31UGS_NDVI_10M_V200.tif
S2A_20210509T103021_31UGS_NDVI_10M_V200
https://services.terrascope.be/download/Sentinel2/NDVI_V2/2021/05/09/S2A_20210509T103021_31UGS_NDVI_V200/S2A_20210509T103021_31UGS_NDVI_10M_V200.tif
S2A_20210512T104021_31UGS_NDVI_10M_V200
https://services.terrascope.be/download/Sentinel2/NDVI_V2/2021/05/12/S2A_20210512T104021_31UGS_NDVI_V200/S2A_20210512T104021_31UGS_NDVI_10M_V200.tif
S2A_20210519T103021_31UGS_NDVI_10M_V210
https://services.terrascope.be/download/Sentinel2/NDVI_V2/2021/05/19/S2A_20210519T103021_31UGS_NDVI_V210/S2A_20210519T103021_31UGS_NDVI_10M_V210.tif
S2A_20210522T104021_31UGS_NDVI_10M_V210
https://service

In [24]:
products_list = list(products)
print(len(products_list))

0


There are lots of optional dependencies depending on where and how the data you are interested in are stored. Here are some of the libraries that you will probably need:

dask - to delay data loading until access

fsspec - to access data from remote storage

pystac - STAC object structures

xarray, rioxarray - data structures

xpystac, stackstac - helper for loading pystac into xarray objects

In [13]:
!pip install adlfs dask 'fsspec[http]' planetary_computer --quiet
!pip install stackstac xarray xpystac zarr --quie

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.8/206.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.1/186.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.4/115.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.6/209.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

In [14]:
import xarray as xr

import pystac

In [27]:
pystac.Collection.authenticate_non_interactive("aquatics", "Welcome!")
from pystac import StacIO

stac_io = StacIO.default()
stac_io.headers = {"Authorization": "<some-auth-header>"}

landsat_item = pystac.Item.from_file(
    "https://services.terrascope.be/download/Sentinel2/NDVI_V2/2021/05/02/S2A_20210502T104021_31UGS_NDVI_V210/S2A_20210502T104021_31UGS_NDVI_10M_V210.tif", stac_io)
xr.open_dataset(landsat_item)

AttributeError: type object 'Collection' has no attribute 'authenticate_non_interactive'